In [3]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\Ilya Jeff\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [32]:
import seaborn as sns
import glob
import torch
import os

import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
sns.set(style="whitegrid")
mpl.rcParams['agg.path.chunksize'] = 10000

#from matplotlib import rc
#rc('text', usetex=True)
mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'


In [31]:
rerun_rounds = ['round1', 'round2', 'round3']
def get_avg_results(path_string):
  """
  For experiments with multiple random training runs, collects all
  training runs and averages their results
  """
  result_list = []
  for rerun_string in rerun_rounds:
    completed_path = path_string.format(rerun_string)
    if os.path.exists(completed_path):
      result_list.append(float(open(completed_path).read()))
  return sum(result_list)/len(result_list) if len(result_list) > 0 else -1.0
  #print(result_list)
  #return np.std(result_list) if len(result_list) > 0 else -1 #sum(result_list)/len(result_list) if len(result_list) > 0 else -1.0

In [33]:
#a = torch.tensor([get_avg_results('reports/dep_rel-6-768-en-distilbert-layer{}.yaml.results/dev.v_entropy'.format(i)) for i in range(1,7)])
#b = torch.tensor([float(open('reports/dep_rel-6-768-en-distilbert-layer{}.yaml.results/dev.v_entropy'.format(i)).read().strip()) for i in range(1,7)])
#c = get_avg_results('reports/layer0.yaml.results/dev.v_entropy')
#print(a)
#print(b)

#for i in range(1,7):
#   print(i)

tensor([1.6499, 1.7123, 1.7404, 1.7500, 1.7595, 1.7668])


In [51]:
def plot_layerwise_zero_baseline(task):
    fig, ax = plt.subplots(figsize=(6,4.5))
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.tick_params(axis='both', which='minor', labelsize=15)
    ax.set_xticks(list(range(1,7)))
    """Makes a plot comparing layerwise bits of info"""
    task_768_singlelayer_ventropy = torch.tensor([get_avg_results('reports/{}-6-768-en-distilbert-layer{}.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,7)])
    task_768_twolayer_ventropy = torch.tensor([get_avg_results('reports/{}-6-768-en-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,7)])
    task_768_embedding_layer_ventropy = get_avg_results('reports/{}-6-768-en-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    print(task_768_singlelayer_ventropy)
    print(task_768_twolayer_ventropy)
    print(task_768_embedding_layer_ventropy)
    x_range = list(range(1,7))
    plt.plot(x_range, task_768_embedding_layer_ventropy-task_768_singlelayer_ventropy, label=r'baselined')
    plt.plot(x_range, task_768_embedding_layer_ventropy-task_768_twolayer_ventropy, label=r'conditional')
    plt.title('6-768-en-distilbert {}'.format(task), fontsize = 12)
    plt.ylabel('Bits', fontsize = 12)
    plt.xlabel('Model layer', fontsize = 12)
    plt.legend()
    plt.tight_layout()
    plt.savefig('6-768-en-distilbert {}.png'.format(task), dpi=200)
    plt.clf()

In [52]:
def plot_accuracy(task):
    fig, ax = plt.subplots(figsize=(6,4.5))
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.tick_params(axis='both', which='minor', labelsize=15)
    ax.set_xticks(list(range(1,7)))
    """Makes a plot comparing layerwise bits of info"""
    label_accuracy = torch.tensor([get_avg_results('reports/{}-6-768-en-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,7)])
    print(label_accuracy)
    x_range = list(range(1,7))
    plt.plot(x_range, label_accuracy)
    plt.title('6-768-en-distilbert {} accuracy'.format(task), fontsize = 12)
    plt.ylabel('Accuracy', fontsize = 12)
    plt.xlabel('Model layer', fontsize = 12)
    plt.tight_layout()
    plt.savefig('6-768-en-distilbert {} accuracy.png'.format(task), dpi=200)
    plt.clf()

In [53]:
plot_layerwise_zero_baseline('dep_rel')
plot_layerwise_zero_baseline('named_entities')
plot_layerwise_zero_baseline('ptb_pos')
plot_layerwise_zero_baseline('upos')
plot_layerwise_zero_baseline('word_sense')


tensor([1.6499, 1.7123, 1.7404, 1.7500, 1.7595, 1.7668])
tensor([1.5143, 1.5517, 1.5526, 1.5625, 1.5648, 1.5689])
1.8580824713655195
tensor([0.6904, 0.7283, 0.7442, 0.7511, 0.7497, 0.7488])
tensor([0.6118, 0.6188, 0.6206, 0.6178, 0.6175, 0.6130])
0.6820613843214373
tensor([0.9823, 1.0595, 1.1062, 1.1246, 1.1411, 1.1573])
tensor([0.8315, 0.8702, 0.8759, 0.8683, 0.8775, 0.8710])
1.0275900919958842
tensor([0.8362, 0.8945, 0.9261, 0.9475, 0.9584, 0.9716])
tensor([0.7201, 0.7389, 0.7455, 0.7467, 0.7502, 0.7502])
0.8696018419484984
tensor([1.4597, 1.5154, 1.5381, 1.5520, 1.5611, 1.5667])
tensor([1.3427, 1.3471, 1.3441, 1.3446, 1.3447, 1.3472])
1.3742971714895635


In [54]:
plot_accuracy('dep_rel')
plot_accuracy('named_entities')
plot_accuracy('ptb_pos')
plot_accuracy('upos')
plot_accuracy('word_sense')


tensor([0.6499, 0.6381, 0.6324, 0.6300, 0.6286, 0.6276])
tensor([0.8902, 0.8854, 0.8822, 0.8814, 0.8816, 0.8811])
tensor([0.7768, 0.7585, 0.7488, 0.7442, 0.7406, 0.7364])
tensor([0.7859, 0.7695, 0.7634, 0.7584, 0.7555, 0.7529])
tensor([0.6712, 0.6610, 0.6577, 0.6543, 0.6519, 0.6526])
